In [1]:
import numpy as np
from deepsudoku import REPO_PATH
import os
import glob
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import tqdm

os.chdir(REPO_PATH)

In [2]:
wanted = ['eval/avg_episodic_length',
  'eval/avg_episodic_return',
  'eval/avg_winrate',
  'charts/avg_episodic_return',
  'charts/avg_episodic_length',
  'losses/value_loss']


def get_df(path):
  event_acc = EventAccumulator(path)
  event_acc.Reload()
  event_acc.Tags()

  df = pd.DataFrame([], columns = ["time_stamp"]+wanted)


  for scalar in wanted:

    for val in event_acc.Scalars(scalar):
      
      
      df.loc[val.step,["time_stamp",scalar]] = [val.wall_time,val.value]
  
  path = Path(path).name
  env = path[:path.find("_")]
  df["env"] = env
  number = path.find("_") + 5
  name = path[path.find("_")+2:path.find("__",number)]
  df["run"] = name
  return df    

dfs = []

for path in tqdm.tqdm(glob.glob("reinforcement_learning_experiment/runs/*")):

  dfs.append(get_df(path))

  0%|          | 0/12 [00:00<?, ?it/s]2023-09-06 17:15:01.094125: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-06 17:15:01.704905: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
100%|██████████| 12/12 [02:20<00:00, 11.69s/it]


In [3]:
data = pd.concat(dfs)
data = data.reset_index(names ="step")
data.to_csv("stats/rl_results.csv")